# Gather Data

In [89]:
import requests
import uuid
import json

In [90]:
def get_auth_token(guid):
    """Fetches auth token from Bird API.
    
    Retrieves an auth token from Bird API by spoofing
    a user login from an iOS device. Uses unique email
    to generate new auth token. Token expires periodically,
    so rerun as necessary.
    
    Args:
        guid: A random 16 Byte GUID of the form
            123E4567-E89B-12D3-A456-426655440000.
    
    Returns:
        A string of the auth token that will be used to
        make future requests to the Bird API.
        
        Returns None if request did not succeed.
    """
    
    url = 'https://api.bird.co/user/login'
    headers = {
        'Device-id': guid,
        'Platform':'ios',
        'Content-Type':'application/json',
    }
    # Reusing guid to ensure uniqueness
    # Only 1 token permitted per unique email address
    payload = {
        'email':'{}@example.com'.format(guid),
    }
    
    r = requests.post(url=url, data=json.dumps(payload), headers=headers)
    
    token = r.json().get('token')
    
    return token

In [91]:
guid = str(uuid.uuid1())

In [92]:
# Get auth token before fetching locations
token = get_auth_token(guid)
assert token is not None

In [93]:
def get_scooter_locations(token, lat, long, radius=1000):
    """Fetches Bird locations given latitude and longitude.
    
    Retrieves Bird locations from Bird API using a valid
    authentication token. List of Bird information is
    retreived by specifing a latitide and longitude centroid
    and a search radius.
    
    Args:
        token: An authentication token for Bird API.
        lat: A latitude for search centroid.
        long: A longitude for search centroid.
        radius: An optional variable for the search radius.
            Units unknown. Default is 1000.
    
    Returns:
        A list of Bird information. Each Bird is represented
        as a JSON object (Python dict) as follows:
        
        {
            "battery_level": 53,
            "captive": false,
            "code": "",
            "id": "9eed3677-12df-4981-a017-e0628a4051fe",
            "location": {
                "latitude": 34.005615,
                "longitude": -118.47679166666667
            }
        }
        
        Returns None if request did not succeed.
    """
    
    url = 'https://api.bird.co/bird/nearby'
    params = {
        'latitude':lat,
        'longitude':long,
        'radius':radius
    }
    headers = {
        'Authorization':'Bird {}'.format(token),
        'Device-id':'{}'.format(guid),
        'App-Version':'3.0.5',
        'Location':json.dumps({
            'latitude':lat,
            'longitude':long,
            'altitude':500,
            'accuracy':100,
            'speed':-1,
            'heading':-1
        })
    }
    r = requests.get(url=url, params=params, headers=headers)
    
    return r.json().get('birds')

In [95]:
# Lat and long of my apartment in Santa Monica
home_location = {'lat':34.005400, 'long':-118.478295}
locs = get_scooter_locations(
    token, 
    lat=home_location['lat'], 
    long=home_location['long'], 
    radius=1000
)
print('{} scooters retrieved.'.format(len(locs)))
print(json.dumps(locs, indent=4, sort_keys=True))

250 scooters retrieved.
[
    {
        "battery_level": 53,
        "captive": false,
        "code": "",
        "id": "9eed3677-12df-4981-a017-e0628a4051fe",
        "location": {
            "latitude": 34.005615,
            "longitude": -118.47677666666665
        }
    },
    {
        "battery_level": 62,
        "captive": false,
        "code": "",
        "id": "59e6ab14-3141-4a2e-a8a6-61753a1887d1",
        "location": {
            "latitude": 34.005085,
            "longitude": -118.480364
        }
    },
    {
        "battery_level": 96,
        "captive": false,
        "code": "",
        "id": "f0a1a568-16db-44f5-ac96-05508848417e",
        "location": {
            "latitude": 34.00677833333334,
            "longitude": -118.47631000000001
        },
        "nest_id": "ab86b751-f0ca-44a8-8617-5127c74d074a"
    },
    {
        "battery_level": 62,
        "captive": false,
        "code": "",
        "id": "8d51268e-ae14-49a5-ae16-39340198ec20",
        "location"